https://devblogs.nvidia.com/deep-learning-self-driving-cars/

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2

tf.python.control_flow_ops = tf

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Convolution2D

Using TensorFlow backend.


In [56]:
import bclone

In [4]:
DATA_DIR_STD = '../bclone-data-standard'
DATA_DIR_NEW = '../my_2018-02-22'

```bash
python preprocess-driving-log.py --src ../my_2018-02-22/driving_log_original.csv --dst ../my_2018-02-22/driving_log.csv
```

In [5]:
def nvidia_model():
    
    model = Sequential()
    
    model.add( Lambda(lambda x: x / 255. - 0.5, input_shape=(160, 320, 3)) )
    model.add( Cropping2D(cropping=((70, 25), (0, 0))) )
    
    model.add( Convolution2D(24, 5, 5, subsample=(2, 2), activation='relu') )
    model.add( Convolution2D(36, 5, 5, subsample=(2, 2), activation='relu') )
    model.add( Convolution2D(48, 5, 5, subsample=(2, 2), activation='relu') )
    
    model.add( Convolution2D(64, 3, 3, subsample=(2, 2), activation='relu') )
    #model.add( Convolution2D(64, 3, 3, subsample=(2, 2), activation='relu') )

    model.add( Flatten() )

    model.add( Dense(100) )
    model.add( Dense(50) )
    model.add( Dense(10) )
    model.add( Dense(1) )
    
    model.compile(loss='mse', optimizer='adam')
    
    return model

In [48]:
def do_training(model, log_df, batch_sz, epochs):
    
    train, valid = train_test_split(log_df, test_size=0.2)
    valid_gen = bclone.data_generator(valid, batch_size=batch_sz, controls=['steering'])
    train_gen = bclone.data_generator(train, batch_size=batch_sz, controls=['steering'])
    
    history = bclone.fit_gen(model, train_gen, valid_gen, train, valid, n_epochs=epochs)
    
    return train, valid, history


def do_training_smart(model, log_dataframes, batch_sizes, epochs):
    
    splitted = [train_test_split(df, test_size=0.2) for df in log_dataframes]
    train_dfs = [t for t, v in splitted]
    valid_dfs = [v for t, v in splitted]
    
    valid_gen = bclone.data_generator_from_muiltiple_sets(valid_dfs, batch_sizes, controls=['steering'])
    train_gen = bclone.data_generator_from_muiltiple_sets(train_dfs, batch_sizes, controls=['steering'])
    
    history = model.fit_generator(
        train_gen,
        samples_per_epoch=sum([len(df) for df in train_dfs])*3,
        validation_data=valid_gen,
        nb_val_samples=sum([len(df) for df in valid_dfs])*3,
        nb_epoch=epochs
    )
    
    return train, valid, history
    

    

In [24]:
log_df_std = bclone.load_and_combine_logs(DATA_DIR_STD)
log_df_new = bclone.load_and_combine_logs(DATA_DIR_NEW)
log_df_all = bclone.load_and_combine_logs(DATA_DIR_STD, DATA_DIR_NEW)

In [35]:
len(log_df_std), len(log_df_new), len(log_df_all)

(8036, 7370, 15406)

In [37]:
log_df_mixin = log_df_std.append(
    shuffle(log_df_new)[:2000], 
    ignore_index=True
)

In [44]:
model = nvidia_model()

#model.load_weights('model.h5')
#model.evaluate_generator(valid_gen_all, len(log_df_valid))

In [38]:
#t1, v1, h1 = do_training(model_1, log_df_std, batch_sz=20, epochs=1)
#t2, v2, h2 = do_training(model_2, log_df_all, batch_sz=20, epochs=3)

In [59]:
t_, v_, h_ = do_training_smart(model, [log_df_std, log_df_new], [40, 8], epochs=2)

Epoch 1/2
36972/36972 [==============================] - 80s - loss: 0.0159 - val_loss: 0.0148
Epoch 2/2
36936/36972 [============================>.] - ETA: 0s - loss: 0.0157

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


37080/36972 [==============================] - 79s - loss: 0.0157 - val_loss: 0.0145


NameError: name 'train' is not defined

In [60]:
model.save('model3.h5')